# Part I:Runge's Phenomenon

> Why high-degree polynomial interpolation fails

Discover the classic problem that motivates the use of splines in approximation theory.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import splinepy
import pygismo as gs

plt.rcParams.update({'figure.figsize': (11, 5), 'font.size': 12,
                     'axes.grid': True, 'grid.alpha': 0.3})

# Workaround: tight_layout can fail on some platforms due to mathtext rendering
_orig_tight_layout = plt.tight_layout
def _safe_tight_layout(**kwargs):
    try:
        _orig_tight_layout(**kwargs)
    except (ValueError, RuntimeError):
        pass
plt.tight_layout = _safe_tight_layout

print(f'splinepy {splinepy.__version__}  |  pygismo {gs.__version__}')

## 1. Runge's Phenomenon

The **Runge function** $\displaystyle f(x)=\frac{1}{1+25\,x^{2}},\quad x\in[-1,1]$

> Move the slider below to see the effect.

In [2]:
def runge(x):
    return 1.0 / (1.0 + 25.0 * x**2)

def lagrange_interp(xn, yn, x):
    n = len(xn)
    L = np.zeros_like(x)
    for i in range(n):
        li = np.ones_like(x)
        for j in range(n):
            if j != i:
                li *= (x - xn[j]) / (xn[i] - xn[j])
        L += yn[i] * li
    return L

x_plot = np.linspace(-1, 1, 800)
y_true = runge(x_plot)

def plot_runge_poly(n=6):
    xn = np.linspace(-1, 1, n + 1)
    yn = runge(xn)
    yp = lagrange_interp(xn, yn, x_plot)
    err = np.max(np.abs(y_true - yp))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.plot(x_plot, y_true, 'k-', lw=2, label=r'$f(x)=\frac{1}{1+25x^2}$')
    ax1.plot(x_plot, yp, 'r-', lw=1.5, label=f'Lagrange poly  n={n}')
    ax1.plot(xn, yn, 'bo', ms=6, zorder=5, label='interp. nodes')
    ax1.set_ylim(-1.5, 2.0)
    ax1.set_title(f'Polynomial interpolation  (degree {n})')
    ax1.legend(loc='upper left', fontsize=10)
    ax1.set_xlabel('$x$')

    ax2.plot(x_plot, np.abs(y_true - yp), 'r-', lw=1.5)
    ax2.set_yscale('log')
    ax2.set_ylim(1e-15, 1e2)
    ax2.set_title(f'Point-wise error   (max = {err:.3e})')
    ax2.set_xlabel('$x$')
    ax2.set_ylabel('$|f(x) - p_n(x)|$')
    plt.tight_layout()
    plt.show()

widgets.interact(plot_runge_poly,
                 n=widgets.IntSlider(min=2, max=30, step=1, value=6,
                                     description='Degree n'));

interactive(children=(IntSlider(value=6, description='Degree n', max=30, min=2), Output()), _dom_classes=('wid…

## 2. Try to Make "Interesting" Curves with High-Order Bézier

In [3]:
from scipy.special import comb

def bernstein(i, n, t):
    """Bernstein basis polynomial B_{i,n}(t)."""
    return comb(n, i, exact=True) * t**i * (1 - t)**(n - i)

def bezier_curve(P, t):
    """Evaluate Bezier curve with control points P at parameters t."""
    n = len(P) - 1
    curve = np.zeros((len(t), 2))
    for i in range(n + 1):
        B = bernstein(i, n, t)
        curve[:, 0] += B * P[i, 0]
        curve[:, 1] += B * P[i, 1]
    return curve

# 8th order Bezier curve (9 control points)
n_bez = 8
t_fine = np.linspace(0, 1, 500)

# Base control points
P_base = np.array([[0, 0], [1, 1], [2, -1], [3, 1], [4, -1],
                   [5, 1], [6, -1], [7, 1], [8, 0]])

def plot_interactive_bezier(point_index=4, point_height=3.0):
    """
    Interactive 8th order Bezier curve.
    Move ONE control point and watch how little the curve responds!
    """
    # Copy base and modify selected point
    P = P_base.copy()
    P[point_index, 1] = point_height
    
    curve_base = bezier_curve(P_base, t_fine)
    curve_modified = bezier_curve(P, t_fine)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: The curves
    ax1.plot(curve_base[:, 0], curve_base[:, 1], 'k--', lw=2, alpha=0.4,
             label='Original curve')
    ax1.plot(P_base[:, 0], P_base[:, 1], 'o', color='gray', ms=7, alpha=0.3)
    
    ax1.plot(curve_modified[:, 0], curve_modified[:, 1], 'b-', lw=3.5, 
             label='Modified curve')
    ax1.plot(P[:, 0], P[:, 1], 'ro--', ms=8, lw=1.5, alpha=0.6, 
             label='Control points')
    
    # Highlight the moved point
    ax1.plot(P[point_index, 0], P[point_index, 1], 'r*', ms=20, zorder=10,
             label=f'Moved point {point_index}')
    
    # Annotate points
    for i in [point_index]:
        ax1.annotate(f'{i}', xy=(P[i, 0], P[i, 1]), xytext=(0, 15), 
                    textcoords='offset points', fontsize=11, ha='center',
                    bbox=dict(boxstyle='circle,pad=0.4', facecolor='yellow', 
                             edgecolor='red', linewidth=2))
    
    ax1.axhline(0, color='k', lw=0.5, alpha=0.3)
    ax1.set_xlabel('$x$', fontsize=12)
    ax1.set_ylabel('$y$', fontsize=12)
    ax1.set_title('8th Order Bézier — Move ONE Point, Curve Barely Responds!', 
                 fontsize=13, fontweight='bold')
    ax1.legend(loc='upper right', fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(-3.5, 3.5)
    ax1.set_xlim(-0.5, 8.5)
    
    # Right: Bernstein basis functions (highlight the moved point's basis)
    ax2.set_title(f'Bernstein Basis $B_{{i,{n_bez}}}(t)$', fontsize=13, fontweight='bold')
    colors = plt.cm.tab10(np.linspace(0, 1, 9))
    for i in range(9):
        B = bernstein(i, n_bez, t_fine)
        if i == point_index:
            ax2.plot(t_fine, B, lw=3.5, color='red', alpha=1.0, 
                    label=f'$B_{{{i},8}}$ (moved point)', zorder=10)
        else:
            ax2.plot(t_fine, B, lw=1.5, color=colors[i], alpha=0.3)
    
    ax2.set_xlabel('$t$', fontsize=12)
    ax2.set_ylabel('Basis weight', fontsize=12)
    ax2.legend(fontsize=10, loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    # Show max weight
    B_max = bernstein(point_index, n_bez, np.linspace(0, 1, 500)).max()
    ax2.annotate(f'Max weight = {B_max:.3f}\n(weak influence!)', 
                xy=(point_index/n_bez, B_max), xytext=(0.7, 0.35), fontsize=10,
                arrowprops=dict(arrowstyle='->', color='red', lw=2),
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.9))
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    displacement = point_height - P_base[point_index, 1]
    curve_change = (curve_modified[:, 1] - curve_base[:, 1]).max()
    
    print(f"\n{'='*65}")
    print(f"Control point {point_index} moved by: {displacement:+.2f}")
    print(f"Maximum curve displacement: {curve_change:.3f}")
    print(f"Response ratio: {curve_change/abs(displacement)*100:.1f}% (should be 100% for local control!)")
    print(f"{'='*65}\n")

# Create simplified interactive widget
print("Interactive Demo: Move ONE control point and see the weak response!\n")

widgets.interact(plot_interactive_bezier,
    point_index=widgets.IntSlider(min=1, max=7, step=1, value=4, 
                                 description='Point #'),
    point_height=widgets.FloatSlider(min=-3, max=3, step=0.2, value=3.0, 
                                    description='Height (y)',
                                    continuous_update=False)
);

Interactive Demo: Move ONE control point and see the weak response!



interactive(children=(IntSlider(value=4, description='Point #', max=7, min=1), FloatSlider(value=3.0, continuo…

## Part II: B-Spline basis

## 1. Property 1: Non-negativity

> Verify that all basis functions stay non-negative everywhere!

In [4]:
def plot_nonnegativity(p=3, n_spans=5):
    internal = list(np.linspace(0, 1, n_spans + 1)[1:-1])
    knots = [0.0]*(p+1) + internal + [1.0]*(p+1)
    n_basis = len(knots) - p - 1
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    xi = np.linspace(0, 1, 1000)
    colors = plt.cm.tab10(np.linspace(0, 1, min(n_basis, 10)))
    
    min_vals = []
    
    # Plot all basis functions
    for i in range(n_basis):
        cps = np.zeros((n_basis, 1))
        cps[i] = 1.0
        bs = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cps)
        vals = bs.evaluate(xi.reshape(-1, 1)).flatten()
        min_vals.append(np.min(vals))
        
        ax1.plot(xi, vals, lw=2, color=colors[i % 10], label=f'$N_{{{i},{p}}}$' if i < 5 else '')
    
    ax1.axhline(0, color='red', ls='--', lw=2, alpha=0.7, label='y = 0 (lower bound)')
    for k in sorted(set(internal)):
        ax1.axvline(k, color='gray', ls=':', alpha=0.4)
    
    ax1.set_title(f'All {n_basis} basis functions (p={p})', fontsize=12)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(-0.1, 1.1)
    ax1.set_xlabel(r'$\xi$')
    ax1.set_ylabel(r'$N_{i,p}(\xi)$')
    ax1.legend(loc='upper right', fontsize=9, ncol=2)
    
    # Bar chart of minimum values
    ax2.bar(range(n_basis), min_vals, color='steelblue', alpha=0.7, edgecolor='black')
    ax2.axhline(0, color='red', ls='--', lw=2, alpha=0.7)
    ax2.set_xlabel('Basis function index $i$')
    ax2.set_ylabel(r'$\min_\xi N_{i,p}(\xi)$')
    ax2.set_title(f'Minimum value of each basis function', fontsize=12)
    ax2.set_ylim(-0.05, 0.05)
    
    plt.suptitle(r'Property 1: Non-negativity  $N_{i,p}(\xi) \geq 0$  $\forall i, \xi$', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print(f'Global minimum across all basis functions: {np.min(min_vals):.2e}')


widgets.interact(plot_nonnegativity,
                 p=widgets.IntSlider(min=0, max=5, value=3, description='Degree p'),
                 n_spans=widgets.IntSlider(min=2, max=12, value=5, description='# spans'));

interactive(children=(IntSlider(value=3, description='Degree p', max=5), IntSlider(value=5, description='# spa…

## 2. Property 2: Local Support
<!-- 
**Key Property**: $N_{i,p}(\xi) = 0$ outside the interval $[\xi_i, \xi_{i+p+1}]$.

Each basis function is non-zero over at most $p+1$ knot spans.

Consequences:
- Moving one control point affects the curve only locally
- Sparse matrices in numerical computation
- Computational efficiency -->

> Move a control point with the slider and see: the curve only changes inside the red region.

In [5]:
def plot_local_support(p=3, n_spans=6, point_index=4, displacement=0.4):
    internal = list(np.linspace(0, 1, n_spans + 1)[1:-1])
    knots = [0.0]*(p+1) + internal + [1.0]*(p+1)
    n_basis = len(knots) - p - 1

    point_index = min(point_index, n_basis - 1)

    xi = np.linspace(0, 1, 1000)
    colors = plt.cm.tab10(np.linspace(0, 1, min(n_basis, 10)))

    # Support region of the moved point's basis function
    support_start = knots[point_index]
    support_end   = knots[point_index + p + 1]
    in_support = (xi >= support_start) & (xi <= support_end)

    # --- Build original and modified curves (2D: x = xi mapped, y from control points) ---
    # Use a sine-like pattern so the curve is visually interesting
    cp_orig = np.column_stack([
        np.linspace(0, 1, n_basis),
        0.3 * np.sin(np.linspace(0, 2*np.pi, n_basis))
    ])
    cp_mod = cp_orig.copy()
    cp_mod[point_index, 1] += displacement

    bs_orig = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cp_orig)
    bs_mod  = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cp_mod)

    curve_orig = bs_orig.evaluate(xi.reshape(-1, 1))
    curve_mod  = bs_mod.evaluate(xi.reshape(-1, 1))

    # ---- Figure: 2 rows, sharing x-axis ----
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), sharex=True,
                                    gridspec_kw={'height_ratios': [1.2, 1]})

    # ===== Top: curve effect =====
    ax1.axvspan(support_start, support_end, alpha=0.12, color='red',
                label=f'Affected region $[\\xi_{{{point_index}}},\\xi_{{{point_index+p+1}}}]$')

    ax1.plot(xi, curve_orig[:, 1], 'k--', lw=1.5, alpha=0.5, label='Original curve')

    # Unaffected parts in blue, affected part in red
    # break into segments to avoid connecting lines across gaps
    xi_out = np.where(~in_support, xi, np.nan)
    xi_in  = np.where(in_support,  xi, np.nan)
    ax1.plot(xi_out, np.where(~in_support, curve_mod[:, 1], np.nan),
             '-', lw=2.5, color='steelblue', label='Unchanged')
    ax1.plot(xi_in, np.where(in_support, curve_mod[:, 1], np.nan),
             '-', lw=3, color='crimson', label='Changed locally')

    # Control polygon
    ax1.plot(cp_mod[:, 0], cp_mod[:, 1], 'o--', color='gray', ms=5, lw=1, alpha=0.4)
    ax1.plot(cp_orig[point_index, 0], cp_orig[point_index, 1],
             'o', color='gray', ms=10, alpha=0.4)
    ax1.plot(cp_mod[point_index, 0], cp_mod[point_index, 1],
             '*', color='crimson', ms=18, zorder=10,
             label=f'Moved $P_{{{point_index}}}$')
    # Arrow from old to new position
    ax1.annotate('', xy=(cp_mod[point_index, 0], cp_mod[point_index, 1]),
                 xytext=(cp_orig[point_index, 0], cp_orig[point_index, 1]),
                 arrowprops=dict(arrowstyle='->', color='crimson', lw=2))

    ax1.set_ylabel('$y$', fontsize=12)
    ax1.set_title(f'Move ONE control point $\\rightarrow$ only LOCAL curve change  '
                  f'(p={p}, {n_spans} spans)', fontsize=13, fontweight='bold')
    ax1.legend(loc='upper right', fontsize=10)
    ax1.grid(True, alpha=0.3)

    # ===== Bottom: basis functions (explain WHY) =====
    ax2.axvspan(support_start, support_end, alpha=0.12, color='red')

    for i in range(n_basis):
        cps_b = np.zeros((n_basis, 1))
        cps_b[i] = 1.0
        bs_b = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cps_b)
        vals = bs_b.evaluate(xi.reshape(-1, 1)).flatten()

        if i == point_index:
            ax2.plot(xi, vals, lw=3, color='crimson',
                     label=f'$N_{{{i},{p}}}$ (moved point)', zorder=10)
        else:
            ax2.plot(xi, vals, lw=1, color=colors[i % 10], alpha=0.25)

    for k in sorted(set(knots)):
        ax2.axvline(k, color='gray', ls=':', alpha=0.3, lw=0.8)

    ax2.set_title(f'$N_{{{point_index},{p}}}$ is non-zero only in '
                  f'$[{support_start:.3f},\\;{support_end:.3f}]$  '
                  f'$\\rightarrow$ only {p+1} spans affected',
                  fontsize=12)
    ax2.set_xlabel(r'$\xi$', fontsize=12)
    ax2.set_ylabel(r'$N_{i,p}(\xi)$', fontsize=12)
    ax2.set_xlim(0, 1)
    ax2.set_ylim(-0.05, 1.1)
    ax2.legend(loc='upper right', fontsize=10)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Summary statistics
    diff = np.abs(curve_mod[:, 1] - curve_orig[:, 1])
    max_change_in  = np.max(diff[in_support])  if np.any(in_support)  else 0
    max_change_out = np.max(diff[~in_support]) if np.any(~in_support) else 0

    print(f'Control point P_{point_index} displaced by {displacement:+.2f}')
    print(f'  Max curve change INSIDE  support: {max_change_in:.4f}')
    print(f'  Max curve change OUTSIDE support: {max_change_out:.2e}  (machine zero)')
    print(f'  Support spans {p+1} knot intervals out of {n_spans} total')

widgets.interact(plot_local_support,
    p=widgets.IntSlider(min=1, max=5, value=3, description='Degree p'),
    n_spans=widgets.IntSlider(min=3, max=10, value=6, description='# spans'),
    point_index=widgets.IntSlider(min=0, max=15, value=4, description='Move point #'),
    displacement=widgets.FloatSlider(min=-0.5, max=0.5, step=0.05, value=0.4,
                                     description='Displacement',
                                     continuous_update=False));

interactive(children=(IntSlider(value=3, description='Degree p', max=5, min=1), IntSlider(value=6, description…

## 3. Property 3: Partition of Unity

<!-- **Key Property**: $\sum_{i} N_{i,p}(\xi) = 1$ for all $\xi$ in the domain.

Consequences:
- Affine invariance: translating control points translates the curve
- No artificial scaling in approximations
- Convex hull property -->

> Watch the sum stay exactly at 1.0!

In [6]:
def plot_partition_of_unity(p=3, n_spans=5, show_individual=True):
    internal = list(np.linspace(0, 1, n_spans + 1)[1:-1])
    knots = [0.0]*(p+1) + internal + [1.0]*(p+1)
    n_basis = len(knots) - p - 1
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    xi = np.linspace(0, 1, 1000)
    colors = plt.cm.tab10(np.linspace(0, 1, min(n_basis, 10)))
    
    total = np.zeros_like(xi)
    
    # Plot individual basis functions
    for i in range(n_basis):
        cps = np.zeros((n_basis, 1))
        cps[i] = 1.0
        bs = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cps)
        vals = bs.evaluate(xi.reshape(-1, 1)).flatten()
        total += vals
        
        if show_individual:
            ax1.plot(xi, vals, lw=1.5, color=colors[i % 10], alpha=0.6)
    
    # Highlight the sum
    ax1.plot(xi, total, 'k-', lw=3, label=r'$\sum_i N_{i,p}(\xi)$', zorder=10)
    ax1.axhline(1.0, color='red', ls='--', lw=2, alpha=0.7, label='y = 1 (target)')
    
    for k in sorted(set(internal)):
        ax1.axvline(k, color='gray', ls=':', alpha=0.4)
    
    ax1.set_title(f'Basis functions and their sum (p={p}, {n_spans} spans)', fontsize=12)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(-0.05, 1.5)
    ax1.set_xlabel(r'$\xi$')
    ax1.set_ylabel(r'$N_{i,p}(\xi)$')
    ax1.legend(loc='upper right', fontsize=10)
    
    # Error from unity
    error = np.abs(total - 1.0)
    ax2.semilogy(xi, error + 1e-17, 'b-', lw=2)
    ax2.set_title(r'Deviation from unity: $|\sum_i N_{i,p}(\xi) - 1|$', fontsize=12)
    ax2.set_xlabel(r'$\xi$')
    ax2.set_ylabel('Absolute error')
    ax2.set_ylim(1e-17, 1e-10)
    ax2.axhline(1e-15, color='red', ls='--', alpha=0.5, label='machine precision')
    ax2.legend()
    
    plt.suptitle(r'Property 2: Partition of Unity  $\sum_i N_{i,p}(\xi) = 1$', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print(f'Max deviation from unity: {np.max(error):.2e}')
    print('✓ Partition of unity verified!' if np.max(error) < 1e-14 else f'Deviation: {np.max(error):.2e}')

widgets.interact(plot_partition_of_unity,
                 p=widgets.IntSlider(min=0, max=5, value=3, description='Degree p'),
                 n_spans=widgets.IntSlider(min=2, max=12, value=5, description='# spans'),
                 show_individual=widgets.Checkbox(value=True, description='Show individual'));

interactive(children=(IntSlider(value=3, description='Degree p', max=5), IntSlider(value=5, description='# spa…

## 4. Property 4: Maximum value

<!-- **Key Property**: Each $N_{i,p}(\xi)$ attains its maximum value exactly once in its support.

For uniform knot vectors with degree $p \geq 1$, the maximum is:
$$\max_\xi N_{i,p}(\xi) = 1$$ -->

> This property ensures good shape control and stability.

In [7]:
def plot_maximum_property(p=3, n_spans=5):
    internal = list(np.linspace(0, 1, n_spans + 1)[1:-1])
    knots = [0.0]*(p+1) + internal + [1.0]*(p+1)
    n_basis = len(knots) - p - 1
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    xi = np.linspace(0, 1, 2000)
    colors = plt.cm.tab10(np.linspace(0, 1, min(n_basis, 10)))
    
    max_vals = []
    max_locs = []
    
    # Plot all basis functions and find maxima
    for i in range(n_basis):
        cps = np.zeros((n_basis, 1))
        cps[i] = 1.0
        bs = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cps)
        vals = bs.evaluate(xi.reshape(-1, 1)).flatten()
        
        max_val = np.max(vals)
        max_idx = np.argmax(vals)
        max_loc = xi[max_idx]
        
        max_vals.append(max_val)
        max_locs.append(max_loc)
        
        ax1.plot(xi, vals, lw=2, color=colors[i % 10], alpha=0.7, label=f'$N_{{{i},{p}}}$' if i < 6 else '')
        ax1.plot(max_loc, max_val, 'o', color=colors[i % 10], ms=8, 
                markeredgecolor='black', markeredgewidth=1.5, zorder=10)
    
    for k in sorted(set(internal)):
        ax1.axvline(k, color='gray', ls=':', alpha=0.4)
    
    ax1.axhline(1.0, color='red', ls='--', lw=1.5, alpha=0.5, label='y = 1')
    ax1.set_title(f'Basis functions with maxima marked (p={p})', fontsize=12)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(-0.05, 1.1)
    ax1.set_xlabel(r'$\xi$')
    ax1.set_ylabel(r'$N_{i,p}(\xi)$')
    ax1.legend(loc='upper right', fontsize=9, ncol=2)
    
    # Maximum values bar chart
    x_pos = np.arange(n_basis)
    ax2.bar(x_pos, max_vals, color='steelblue', alpha=0.7, edgecolor='black', label='max value')
    ax2.axhline(1.0, color='red', ls='--', lw=2, alpha=0.7, label='y = 1 (uniform case)')
    ax2.set_xlabel('Basis function index $i$')
    ax2.set_ylabel(r'$\max_\xi N_{i,p}(\xi)$')
    ax2.set_title(f'Maximum value of each basis function', fontsize=12)
    ax2.set_ylim(0, 1.15)
    ax2.legend()
    
    plt.suptitle(r'Property 5: Each $N_{i,p}$ has unique maximum', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print(f'Maximum values: {[f"{v:.4f}" for v in max_vals]}')
    print(f'All maxima = 1.0: {all(abs(v - 1.0) < 1e-10 for v in max_vals)}')

widgets.interact(plot_maximum_property,
                 p=widgets.IntSlider(min=1, max=5, value=3, description='Degree p'),
                 n_spans=widgets.IntSlider(min=2, max=10, value=5, description='# spans'));

interactive(children=(IntSlider(value=3, description='Degree p', max=5, min=1), IntSlider(value=5, description…

## 5. Property 5: Translation Invariance (Uniform Knot Vectors)


> **Row 1** — Original basis functions: interior ones highlighted in color, boundary ones in gray.

> **Row 2** — Shift each interior function to local coordinate $t = \xi - \xi_i$: uniform knots → all curves overlap perfectly; non-uniform → shapes differ.

In [8]:
def plot_translation_invariance(p=3, n_spans=8):
    """Demonstrate translation invariance: uniform vs non-uniform knot vectors.

    2 rows × 2 columns:
      Row 1: All basis functions in original positions
      Row 2: Interior functions shifted to local coordinate t = ξ - ξ_i
    """
    xi = np.linspace(0, 1, 2000)

    def eval_basis_on_grid(knots, deg, idx, grid):
        """Evaluate N_{idx, deg} on arbitrary parameter values; zero outside domain."""
        n_b = len(knots) - deg - 1
        cps = np.zeros((n_b, 1)); cps[idx] = 1.0
        bs = splinepy.BSpline(degrees=[deg], knot_vectors=[knots], control_points=cps)
        mask = (grid >= knots[0]) & (grid <= knots[-1])
        vals = np.zeros_like(grid)
        if np.any(mask):
            vals[mask] = bs.evaluate(grid[mask].reshape(-1, 1)).flatten()
        return vals

    # ── Knot vectors ──
    internal_u = list(np.linspace(0, 1, n_spans + 1)[1:-1])
    knots_u = [0.0]*(p+1) + internal_u + [1.0]*(p+1)
    n_u = len(knots_u) - p - 1
    h = 1.0 / n_spans

    rng = np.random.RandomState(42)
    internal_nu = list(np.sort(rng.uniform(0.05, 0.95, n_spans - 1)))
    knots_nu = [0.0]*(p+1) + internal_nu + [1.0]*(p+1)
    n_nu = len(knots_nu) - p - 1

    # ── Interior indices (full support, not affected by boundary clamping) ──
    is_u, ie_u = p, max(n_u - p, p + 1)
    is_nu, ie_nu = p, max(n_nu - p, p + 1)

    # ── Colors ──
    n_int_max = max(ie_u - is_u, ie_nu - is_nu, 1)
    int_colors = plt.cm.Dark2(np.linspace(0, 0.9, n_int_max))

    # ── Local coordinate grids ──
    support_u = (p + 1) * h
    t_u = np.linspace(0, support_u, 1000)

    max_support_nu = max(knots_nu[i + p + 1] - knots_nu[i]
                         for i in range(is_nu, ie_nu))
    t_nu = np.linspace(0, max_support_nu, 1000)

    # ═══════ Figure: 2 rows × 2 columns ═══════
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))

    configs = [
        (knots_u,  n_u,  is_u,  ie_u,  t_u,  'Uniform',     'crimson'),
        (knots_nu, n_nu, is_nu, ie_nu, t_nu, 'Non-uniform', 'steelblue'),
    ]

    for col, (knots, n_b, is_, ie_, t_local, label, accent) in enumerate(configs):
        n_int = ie_ - is_

        # ━━━ Row 0: original positions ━━━
        ax = axes[0, col]
        for i in range(n_b):
            vals = eval_basis_on_grid(knots, p, i, xi)
            if is_ <= i < ie_:
                ci = i - is_
                ax.plot(xi, vals, lw=2.5, color=int_colors[ci], alpha=0.85,
                        label=rf'$N_{{{i},{p}}}$' if ci < 6 else '')
            else:
                ax.plot(xi, vals, lw=1, color='silver', alpha=0.4)

        # Knot markers as triangles on x-axis
        unique_knots = sorted(set(knots))
        ax.plot(unique_knots, [-0.04]*len(unique_knots), '^',
                color=accent, ms=7, clip_on=False, zorder=10)

        ax.set_title(f'{label} knots  (h {"= " + f"{h:.3f}" if col == 0 else "varies"})',
                     fontsize=13, fontweight='bold', color=accent)
        ax.set_xlim(-0.01, 1.01)
        ax.set_ylim(-0.06, 1.12)
        ax.set_ylabel(r'$N_{i,p}(\xi)$', fontsize=11)
        ax.legend(fontsize=8, ncol=2, loc='upper right')
        ax.grid(True, alpha=0.15)

        # ━━━ Row 1: shifted to local coordinate t = ξ − ξ_i ━━━
        ax = axes[1, col]

        for ci, i in enumerate(range(is_, ie_)):
            eval_pts = knots[i] + t_local
            vals = eval_basis_on_grid(knots, p, i, eval_pts)
            ax.plot(t_local, vals, lw=2.5, color=int_colors[ci], alpha=0.7,
                    label=rf'$N_{{{i},{p}}}(\xi_i + t)$' if ci < 5 else '')

        ax.set_xlim(t_local[0] - 0.005, t_local[-1] * 1.05)
        ax.set_ylim(-0.05, 1.12)
        ax.set_xlabel(r'Local coordinate  $t = \xi - \xi_i$', fontsize=11)
        ax.set_ylabel(r'$N_{i,p}(\xi_i + t)$', fontsize=11)
        ax.legend(fontsize=8, loc='upper right')
        ax.grid(True, alpha=0.15)

        if col == 0:
            ax.set_title('Shifted to local coord  →  PERFECT overlap',
                         fontsize=12, fontweight='bold', color=accent)
            ax.text(0.5, 0.48, 'IDENTICAL', transform=ax.transAxes,
                    fontsize=30, ha='center', va='center', color=accent,
                    fontweight='bold', alpha=0.12, rotation=20)
        else:
            ax.set_title('Shifted to local coord  →  shapes DIFFER',
                         fontsize=12, fontweight='bold', color=accent)
            ax.text(0.5, 0.48, 'DIFFERENT', transform=ax.transAxes,
                    fontsize=30, ha='center', va='center', color=accent,
                    fontweight='bold', alpha=0.12, rotation=20)

    plt.suptitle(
        rf'Translation Invariance  ($p = {p}$, {n_spans} spans):  '
        rf'uniform $\Rightarrow$ $N_{{i,p}}(\xi) = N_{{j,p}}(\xi - (j-i)h)$',
        fontsize=15, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()

    # ── Numerical summary ──
    print(f'UNIFORM (h = {h:.4f}):')
    print(f'  All {ie_u - is_u} interior basis functions are identical up to translation.')
    print(f'  N_{{i,p}}(ξ) depends only on the local coordinate (ξ − ξ_i)/h,')
    print(f'  which is the same for all interior i when h is constant.\n')

    spacings = [f'{knots_nu[i+1]-knots_nu[i]:.3f}'
                for i in range(p, len(knots_nu)-p-2)]
    print(f'NON-UNIFORM:')
    print(f'  Knot spacings vary: h_i = {spacings}')
    print(f'  Each basis function sees different local knot ratios → DIFFERENT shapes.')

widgets.interact(plot_translation_invariance,
    p=widgets.IntSlider(min=1, max=5, value=3, description='Degree p'),
    n_spans=widgets.IntSlider(min=4, max=12, value=8, description='# spans'));

interactive(children=(IntSlider(value=3, description='Degree p', max=5, min=1), IntSlider(value=8, description…

## 6. Derivative Recursion

> Select a basis function to see its derivative decomposed into lower-degree components!

In [9]:
def plot_derivative_recursion(p=3, n_spans=5, basis_index=3):
    """Visualize the full recursive derivative cascade:
    each differentiation reduces degree by 1 and adds one more component."""
    internal = list(np.linspace(0, 1, n_spans + 1)[1:-1])
    knots = [0.0]*(p+1) + internal + [1.0]*(p+1)
    n_basis = len(knots) - p - 1
    basis_index = min(basis_index, n_basis - 1)

    xi = np.linspace(0, 1, 1000)

    # Helper: evaluate N_{idx, deg} on the same knot vector
    def eval_basis(idx, deg):
        n_b = len(knots) - deg - 1
        if idx < 0 or idx >= n_b:
            return np.zeros_like(xi)
        cps = np.zeros((n_b, 1))
        cps[idx] = 1.0
        bs = splinepy.BSpline(degrees=[deg], knot_vectors=[knots], control_points=cps)
        return bs.evaluate(xi.reshape(-1, 1)).flatten()

    # ── Recursively compute coefficients at each derivative level ──
    # coeffs[k] = {basis_index: weight} at degree (p - k)
    # Level 0: just the original basis function
    coeffs = [{basis_index: 1.0}]

    max_level = min(p, 3)  # show up to 3 levels of differentiation

    for k in range(1, max_level + 1):
        prev = coeffs[k - 1]
        deg = p - (k - 1)  # degree at previous level
        new_coeffs = {}
        for idx, c in prev.items():
            denom_a = knots[idx + deg] - knots[idx]
            denom_b = knots[idx + deg + 1] - knots[idx + 1]
            alpha = deg / denom_a if denom_a > 1e-14 else 0.0
            beta  = deg / denom_b if denom_b > 1e-14 else 0.0
            if alpha != 0:
                new_coeffs[idx] = new_coeffs.get(idx, 0.0) + c * alpha
            if beta != 0:
                new_coeffs[idx + 1] = new_coeffs.get(idx + 1, 0.0) - c * beta
        coeffs.append(new_coeffs)

    # ── Plot: one row per derivative level ──
    n_rows = max_level + 1
    fig, axes = plt.subplots(n_rows, 1, figsize=(14, 3.2 * n_rows), sharex=True)
    if n_rows == 1:
        axes = [axes]

    component_colors = ['crimson', 'forestgreen', 'royalblue', 'darkorange', 'purple']

    # Reference spline for numerical derivatives
    cps_p = np.zeros((n_basis, 1))
    cps_p[basis_index] = 1.0
    bs_p = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cps_p)

    for k in range(n_rows):
        ax = axes[k]
        deg = p - k

        # Faint background: all degree-deg basis functions
        n_b = len(knots) - deg - 1
        for i in range(n_b):
            v = eval_basis(i, deg)
            ax.plot(xi, v, lw=0.5, color='gray', alpha=0.10)

        # Actual k-th derivative (black dashed, for verification)
        if k == 0:
            actual = bs_p.evaluate(xi.reshape(-1, 1)).flatten()
        else:
            actual = bs_p.derivative(xi.reshape(-1, 1), [k]).flatten()

        deriv_label = (rf'$N_{{{basis_index},{p}}}$' if k == 0
                       else rf'$d^{{{k}}}N_{{{basis_index},{p}}}/d\xi^{{{k}}}$')
        ax.plot(xi, actual, 'k--', lw=2, alpha=0.5, label=deriv_label + ' (numerical)')

        # Weighted components from the recursion
        sorted_indices = sorted(coeffs[k].keys())
        for ci, idx in enumerate(sorted_indices):
            c = coeffs[k][idx]
            vals = eval_basis(idx, deg)
            weighted = c * vals

            color = component_colors[ci % len(component_colors)]
            ax.fill_between(xi, 0, weighted, color=color, alpha=0.08)
            sign = '+' if c >= 0 else ''
            ax.plot(xi, weighted, lw=2.5, color=color,
                    label=rf'${sign}{c:.2f} \cdot N_{{{idx},{deg}}}$')

        ax.axhline(0, color='k', lw=0.3)
        for kv in sorted(set(knots)):
            ax.axvline(kv, color='gray', ls=':', alpha=0.15, lw=0.5)

        # Row title with recursion arrow
        n_comp = len(coeffs[k])
        if k == 0:
            title = rf'Level 0  —  $N_{{{basis_index},{p}}}(\xi)$   [degree {deg}, {n_comp} component]'
        else:
            title = (rf'Level {k}  —  $d^{{{k}}}/d\xi^{{{k}}}$  =  '
                     rf'linear combination of {n_comp} degree-{deg} B-splines')

        ax.set_title(title, fontsize=12, fontweight='bold', loc='left')
        ax.set_ylabel(rf'deg {deg}', fontsize=12, fontweight='bold')
        ax.legend(fontsize=9, loc='upper right', ncol=min(n_comp + 1, 5))
        ax.grid(True, alpha=0.15)

        # Draw a downward arrow between rows to emphasize recursion
        if k < n_rows - 1:
            ax.annotate(r'$\frac{d}{d\xi}$   ↓   degree drops by 1',
                        xy=(0.5, 0), xycoords='axes fraction',
                        xytext=(0.5, -0.18), textcoords='axes fraction',
                        fontsize=11, ha='center', color='crimson', fontweight='bold',
                        arrowprops=dict(arrowstyle='->', color='crimson', lw=2))

    axes[-1].set_xlabel(r'$\xi$', fontsize=12)

    plt.suptitle(
        rf'Derivative Recursion Cascade for $N_{{{basis_index},{p}}}$: '
        rf'each $d/d\xi$ lowers degree by 1',
        fontsize=14, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()

    # Print the full recursion chain
    print('Recursion chain:')
    for k in range(n_rows):
        deg = p - k
        terms = [f'{c:+.3f}·N_{{{idx},{deg}}}' for idx, c in sorted(coeffs[k].items())]
        if k == 0:
            prefix = f'  N_{{{basis_index},{p}}}'
        else:
            prefix = f'  d^{k}/dξ^{k} N_{{{basis_index},{p}}}'
        print(f'{prefix}  =  {" ".join(terms)}   [degree {deg}, {len(coeffs[k])} terms]')
    print(f'\n  Each level: differentiate → degree drops by 1, #components increases by 1')

widgets.interact(plot_derivative_recursion,
    p=widgets.IntSlider(min=2, max=5, value=3, description='Degree p'),
    n_spans=widgets.IntSlider(min=3, max=10, value=5, description='# spans'),
    basis_index=widgets.IntSlider(min=0, max=15, value=3, description='Basis index'));

interactive(children=(IntSlider(value=3, description='Degree p', max=5, min=2), IntSlider(value=5, description…

## 7. Interpolation Property at Repeated Knots
> The demo shows both cases: the interior knot (multiplicity $p$, red) and boundary knots (multiplicity $p+1$, blue).

In [10]:
def plot_interpolation_at_repeated_knots(p=3):
    """Demonstrate interpolation at repeated knots:
    - Interior knot with mult = p  -> one basis function = 1
    - Boundary knot with mult = p+1 -> endpoint interpolation
    Show the local knot span structure and the curve passing through control points."""

    # -- Build knot vector: interior knot at 0.5 with multiplicity p --
    knots = [0.0]*(p+1) + [0.25] + [0.5]*p + [0.75] + [1.0]*(p+1)
    n_basis = len(knots) - p - 1

    xi = np.linspace(0, 1, 2000)
    colors = plt.cm.tab10(np.linspace(0, 1, min(n_basis, 10)))

    # -- Find the interpolating basis function at xi = 0.5 --
    xi_pt = 0.5
    vals_at_knot = []
    basis_curves = []
    for i in range(n_basis):
        cps = np.zeros((n_basis, 1)); cps[i] = 1.0
        bs = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cps)
        basis_curves.append(bs.evaluate(xi.reshape(-1, 1)).flatten())
        vals_at_knot.append(bs.evaluate(np.array([[xi_pt]])).flatten()[0])
    vals_at_knot = np.array(vals_at_knot)
    interp_idx = int(np.argmax(vals_at_knot))  # the one that equals 1

    # Its local knot span: knots[interp_idx] ... knots[interp_idx + p + 1]
    local_span = [knots[interp_idx + j] for j in range(p + 2)]

    # -- Build a 2D curve to show geometric consequence --
    cp_y = 0.4 * np.sin(np.linspace(0, 2*np.pi, n_basis)) + 0.5
    cp_2d = np.column_stack([np.linspace(0, 1, n_basis), cp_y])
    bs_curve = splinepy.BSpline(degrees=[p], knot_vectors=[knots], control_points=cp_2d)
    curve_pts = bs_curve.evaluate(xi.reshape(-1, 1))
    pt_at_knot = bs_curve.evaluate(np.array([[xi_pt]])).flatten()
    pt_at_0 = bs_curve.evaluate(np.array([[0.0]])).flatten()
    pt_at_1 = bs_curve.evaluate(np.array([[1.0]])).flatten()

    # ========== Figure ==========
    fig, axes = plt.subplots(2, 1, figsize=(15, 11),
                              gridspec_kw={'height_ratios': [1.2, 1]})

    # === Top: Basis functions ===
    ax = axes[0]
    supp_lo, supp_hi = local_span[0], local_span[-1]
    ax.axvspan(supp_lo, supp_hi, alpha=0.08, color='crimson')

    for i in range(n_basis):
        is_interp = (i == interp_idx)
        lw = 3.5 if is_interp else 1.2
        al = 1.0 if is_interp else 0.3
        color = 'crimson' if is_interp else colors[i % 10]
        zorder = 10 if is_interp else 1
        label = (rf'$N_{{{i},{p}}}$  (interpolates at $\xi={xi_pt}$)' if is_interp
                 else (rf'$N_{{{i},{p}}}$' if i < 5 else ''))
        ax.plot(xi, basis_curves[i], lw=lw, color=color, alpha=al,
                zorder=zorder, label=label)

        ms = 16 if is_interp else 6
        marker = '*' if is_interp else 'o'
        ec = 'black' if is_interp else 'none'
        ax.plot(xi_pt, vals_at_knot[i], marker, color=color, ms=ms,
                markeredgecolor=ec, markeredgewidth=1.5, zorder=11)

    # Boundary interpolation markers
    ax.plot(0.0, 1.0, '*', color='steelblue', ms=16, markeredgecolor='black',
            markeredgewidth=1.5, zorder=11)
    ax.plot(1.0, 1.0, '*', color='steelblue', ms=16, markeredgecolor='black',
            markeredgewidth=1.5, zorder=11)

    ax.axvline(xi_pt, color='red', ls='--', lw=2, alpha=0.5)
    for kv in sorted(set(knots)):
        ax.axvline(kv, color='gray', ls=':', alpha=0.2, lw=0.5)

    # -- Annotate local knot span (plain text, no \underbrace) --
    span_vals = ', '.join(f'{v:.2f}' for v in local_span)
    ann_text = (rf'Local knot span of $N_{{{interp_idx},{p}}}$:' + '\n'
                + '{' + span_vals + '}' + '\n'
                + rf'= $\{{\xi_{{j-1}},\;\xi_j,\;\ldots,\;\xi_j,\;\xi_{{j+1}}\}}$ with {p} repeated $\xi_j$')
    ax.annotate(
        ann_text,
        xy=(xi_pt, 1.0), xytext=(xi_pt + 0.05, 1.15),
        fontsize=10, color='crimson', fontweight='bold',
        arrowprops=dict(arrowstyle='->', color='crimson', lw=1.5),
        bbox=dict(boxstyle='round,pad=0.4', fc='white', ec='crimson', alpha=0.9))

    # Annotate boundary
    ax.annotate(
        rf'Boundary: mult $= p+1 = {p+1}$',
        xy=(0.0, 1.0), xytext=(0.02, 0.75),
        fontsize=10, color='steelblue', fontweight='bold',
        arrowprops=dict(arrowstyle='->', color='steelblue', lw=1.5))

    ax.set_xlim(-0.02, 1.02)
    ax.set_ylim(-0.05, 1.32)
    ax.set_xlabel(r'$\xi$', fontsize=12)
    ax.set_ylabel(r'$N_{i,p}(\xi)$', fontsize=12)
    ax.set_title(
        rf'Basis functions (p = {p}): $N_{{{interp_idx},{p}}}(\xi={xi_pt}) = 1$ '
        rf'because its local knot span has {p} repeated knots at $\xi_j$',
        fontsize=13, fontweight='bold')
    ax.legend(fontsize=9, loc='center right', ncol=1)
    ax.grid(True, alpha=0.15)

    # === Bottom: Curve with control points -- geometric consequence ===
    ax2 = axes[1]
    ax2.plot(curve_pts[:, 0], curve_pts[:, 1], 'k-', lw=2.5, label='B-spline curve', zorder=5)
    ax2.plot(cp_2d[:, 0], cp_2d[:, 1], 'o--', color='gray', ms=8, lw=1.2, alpha=0.5,
             label='Control polygon', zorder=3)

    for i in range(n_basis):
        ax2.annotate(rf'$P_{{{i}}}$', xy=(cp_2d[i, 0], cp_2d[i, 1]),
                     xytext=(0, 10), textcoords='offset points',
                     fontsize=9, ha='center', color='gray', alpha=0.7)

    # Interior: curve passes through P_{interp_idx} at xi = 0.5
    ax2.plot(pt_at_knot[0], pt_at_knot[1], '*', color='crimson', ms=20,
             markeredgecolor='black', markeredgewidth=1.5, zorder=12,
             label=rf'$C({xi_pt}) = P_{{{interp_idx}}}$ (interior, mult $= p$)')
    ax2.annotate(
        rf'$C({xi_pt}) = P_{{{interp_idx}}}$',
        xy=(pt_at_knot[0], pt_at_knot[1]),
        xytext=(pt_at_knot[0] + 0.08, pt_at_knot[1] + 0.08),
        fontsize=12, color='crimson', fontweight='bold',
        arrowprops=dict(arrowstyle='->', color='crimson', lw=2),
        bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='crimson', alpha=0.9))

    # Boundary
    ax2.plot(pt_at_0[0], pt_at_0[1], '*', color='steelblue', ms=20,
             markeredgecolor='black', markeredgewidth=1.5, zorder=12,
             label=rf'$C(0) = P_0$ (boundary, mult $= p+1$)')
    ax2.plot(pt_at_1[0], pt_at_1[1], '*', color='steelblue', ms=20,
             markeredgecolor='black', markeredgewidth=1.5, zorder=12,
             label=rf'$C(1) = P_{{{n_basis-1}}}$ (boundary, mult $= p+1$)')
    ax2.annotate(rf'$C(0) = P_0$', xy=(pt_at_0[0], pt_at_0[1]),
                 xytext=(pt_at_0[0] + 0.06, pt_at_0[1] - 0.08),
                 fontsize=11, color='steelblue', fontweight='bold',
                 arrowprops=dict(arrowstyle='->', color='steelblue', lw=1.5))
    ax2.annotate(rf'$C(1) = P_{{{n_basis-1}}}$', xy=(pt_at_1[0], pt_at_1[1]),
                 xytext=(pt_at_1[0] - 0.15, pt_at_1[1] - 0.10),
                 fontsize=11, color='steelblue', fontweight='bold',
                 arrowprops=dict(arrowstyle='->', color='steelblue', lw=1.5))

    ax2.set_xlabel('$x$', fontsize=12)
    ax2.set_ylabel('$y$', fontsize=12)
    ax2.set_title(
        r'Geometric consequence: curve passes through control points at interpolating knots',
        fontsize=13, fontweight='bold')
    ax2.legend(fontsize=9, loc='best')
    ax2.grid(True, alpha=0.15)
    ax2.set_aspect('auto')

    plt.tight_layout()
    plt.show()

    # -- Summary --
    print(f'Knot vector: {[f"{k:.2f}" for k in knots]}')
    print(f'Degree p = {p}, n_basis = {n_basis}\n')

    print(f'INTERIOR knot xi_j = {xi_pt} (multiplicity = {p}):')
    print(f'  Local knot span of N_{{{interp_idx},{p}}}: {[f"{v:.2f}" for v in local_span]}')
    print(f'  Structure: {{xi_{{j-1}}, xi_j, ..., xi_j, xi_{{j+1}}}} with {p} repeated xi_j')
    print(f'  N_{{{interp_idx},{p}}}({xi_pt}) = {vals_at_knot[interp_idx]:.6f}')
    others = [f'N_{{{i},{p}}}={vals_at_knot[i]:.1e}' for i in range(n_basis)
              if i != interp_idx and abs(vals_at_knot[i]) > 1e-16]
    if others:
        print(f'  Non-zero others: {", ".join(others)}')
    else:
        print(f'  All other N_{{i,{p}}}({xi_pt}) = 0')
    print(f'  -> Curve passes through P_{{{interp_idx}}} = ({cp_2d[interp_idx, 0]:.3f}, {cp_2d[interp_idx, 1]:.3f})')

    print(f'\nBOUNDARY knots (multiplicity = {p+1}):')
    print(f'  xi = 0: N_{{0,{p}}}(0) = 1  -> C(0) = P_0 = ({cp_2d[0, 0]:.3f}, {cp_2d[0, 1]:.3f})')
    print(f'  xi = 1: N_{{{n_basis-1},{p}}}(1) = 1  -> C(1) = P_{{{n_basis-1}}} = ({cp_2d[-1, 0]:.3f}, {cp_2d[-1, 1]:.3f})')

widgets.interact(plot_interpolation_at_repeated_knots,
    p=widgets.IntSlider(min=2, max=5, value=3, description='Degree p'));

interactive(children=(IntSlider(value=3, description='Degree p', max=5, min=2), Output()), _dom_classes=('widg…

## Part III: B-Spline Curves

> Adjust control point positions to see how they influence the curve shape.

In [11]:
def plot_interactive_curve(p1_y=2.0, p2_x=3.0, p2_y=3.0, p3_y=1.0):
    cp = np.array([[0,0],[1,p1_y],[p2_x,p2_y],[4,p3_y],[5,2.5],[6,0.5]], dtype=float)
    
    curve = splinepy.BSpline(
        degrees=[3],
        knot_vectors=[[0,0,0,0, 0.33, 0.67, 1,1,1,1]],
        control_points=cp)
    
    t = np.linspace(0, 1, 400).reshape(-1, 1)
    pts = curve.evaluate(t)
    
    fig, ax = plt.subplots(figsize=(11, 6))
    ax.plot(cp[:,0], cp[:,1], 'r--o', lw=1.5, ms=10, label='Control polygon', zorder=3)
    ax.plot(pts[:,0], pts[:,1], 'b-', lw=3, label='B-spline curve', zorder=2)
    
    for i,(x,y) in enumerate(cp):
        ax.annotate(f'$P_{i}$', (x,y), textcoords='offset points',
                    xytext=(8,8), fontsize=12, fontweight='bold')
    
    ax.legend(fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1, 7)
    ax.set_ylim(-1, 5)
    ax.set_title('Interactive B-spline Curve', fontsize=13)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

widgets.interact(plot_interactive_curve,
    p1_y=widgets.FloatSlider(min=-2, max=5, value=2.0, step=0.1, description='P₁ y'),
    p2_x=widgets.FloatSlider(min=1, max=5, value=3.0, step=0.1, description='P₂ x'),
    p2_y=widgets.FloatSlider(min=-2, max=5, value=3.0, step=0.1, description='P₂ y'),
    p3_y=widgets.FloatSlider(min=-2, max=5, value=1.0, step=0.1, description='P₃ y'));

interactive(children=(FloatSlider(value=2.0, description='P₁ y', max=5.0, min=-2.0), FloatSlider(value=3.0, de…

## Part IV: B-Spline Surface
> Adjust the height of the center control point to see how it affects the surface.

In [12]:
def plot_interactive_surface(center_z=1.0):
    # Create a 3x3 control point grid
    cp_grid = np.array([
        [0, 0, 0], [1, 0, 0.5], [2, 0, 0],
        [0, 1, 0.3], [1, 1, center_z], [2, 1, 0.3],
        [0, 2, 0], [1, 2, 0.5], [2, 2, 0]
    ], dtype=float)
    
    surf = splinepy.BSpline(
        degrees=[2, 2],
        knot_vectors=[[0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1]],
        control_points=cp_grid)
    
    # Evaluate surface
    n_eval = 30
    u = np.linspace(0, 1, n_eval)
    v = np.linspace(0, 1, n_eval)
    U, V = np.meshgrid(u, v)
    uv_pts = np.column_stack([U.ravel(), V.ravel()])
    xyz = surf.evaluate(uv_pts)
    X = xyz[:, 0].reshape(n_eval, n_eval)
    Y = xyz[:, 1].reshape(n_eval, n_eval)
    Z = xyz[:, 2].reshape(n_eval, n_eval)
    
    # Plot
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8, edgecolor='k', linewidth=0.2)
    
    # Control points
    cp_3x3 = cp_grid.reshape(3, 3, 3)
    ax.scatter(cp_3x3[:, :, 0], cp_3x3[:, :, 1], cp_3x3[:, :, 2], 
               c='red', s=100, edgecolor='black', linewidth=2, zorder=10)
    
    # Highlight center point
    ax.scatter([1], [1], [center_z], c='lime', s=200, edgecolor='black', 
               linewidth=3, zorder=11, marker='*', label=f'Center (z={center_z:.2f})')
    
    # Control net
    for i in range(3):
        ax.plot(cp_3x3[i, :, 0], cp_3x3[i, :, 1], cp_3x3[i, :, 2], 'r--', lw=1, alpha=0.5)
    for j in range(3):
        ax.plot(cp_3x3[:, j, 0], cp_3x3[:, j, 1], cp_3x3[:, j, 2], 'r--', lw=1, alpha=0.5)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_zlim(-0.5, 2.5)
    ax.set_title(f'Interactive B-Spline Surface (center z={center_z:.2f})', fontsize=13)
    ax.legend()
    plt.tight_layout()
    plt.show()

widgets.interact(plot_interactive_surface,
    center_z=widgets.FloatSlider(min=-1.0, max=3.0, value=1.0, step=0.1,
                                 description='Center Z'));
                                 


interactive(children=(FloatSlider(value=1.0, description='Center Z', max=3.0, min=-1.0), Output()), _dom_class…

## Part V: NURBS


## 1. Effect of Weights
> Adjust the weight to see how it pulls or pushes the curve.

In [13]:
def plot_nurbs_weight_effect(w1=1.0):
    cp = np.array([[0, 0], [1, 2], [3, 0]], dtype=float)
    weights = np.array([1.0, w1, 1.0])
    
    nurbs = splinepy.NURBS(
        degrees=[2], knot_vectors=[[0, 0, 0, 1, 1, 1]],
        control_points=cp, weights=weights)
    
    bspline = splinepy.BSpline(
        degrees=[2], knot_vectors=[[0, 0, 0, 1, 1, 1]], control_points=cp)
    
    t = np.linspace(0, 1, 200).reshape(-1, 1)
    pts_nurbs = nurbs.evaluate(t)
    pts_bs = bspline.evaluate(t)
    
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.plot(cp[:, 0], cp[:, 1], 'ko--', lw=1.5, ms=8, alpha=0.5, label='Control polygon')
    ax.plot(pts_bs[:, 0], pts_bs[:, 1], 'g:', lw=2, label='B-spline (w=1)', alpha=0.7)
    ax.plot(pts_nurbs[:, 0], pts_nurbs[:, 1], 'b-', lw=3, label=f'NURBS (w₁={w1:.2f})')
    ax.plot(cp[1, 0], cp[1, 1], 'go', ms=15, markeredgecolor='darkgreen', 
            markeredgewidth=2, label=f'$P_1$ (w₁={w1:.2f})', zorder=5)
    
    ax.set_aspect('equal')
    ax.legend(fontsize=10)
    ax.set_xlim(-0.5, 3.5)
    ax.set_ylim(-0.5, 2.8)
    ax.set_title(f'Effect of Weight on NURBS Curve', fontsize=13)
    ax.grid(True, alpha=0.3)
    plt.tight_layout(); plt.show()
    

widgets.interact(plot_nurbs_weight_effect,
    w1=widgets.FloatSlider(min=0.1, max=5.0, value=1.0, step=0.1, description='Weight w₁'));

interactive(children=(FloatSlider(value=1.0, description='Weight w₁', max=5.0, min=0.1), Output()), _dom_class…

## 2. Refinement Strategies
1. **h-refinement**: Insert knots -> more elements
2. **p-refinement**: Elevate degree -> higher order
3. **k-refinement**: h + p together

In [14]:

def runge(x):
    return 1.0 / (1.0 + 25.0 * x**2)

def refinement_demo(strategy='h-refine', level=2):
    p_base = 3
    kv_base = [0.0]*(p_base+1) + [0.5] + [1.0]*(p_base+1)

    if strategy == 'h-refine':
        kv_gs = gs.nurbs.gsKnotVector(kv_base, p_base)
        basis = gs.nurbs.gsBSplineBasis(kv_gs)
        for _ in range(level): basis.uniformRefine(1)
        p_cur = p_base
        title = f'h-refine (p={p_cur}, n={basis.size()})'
    elif strategy == 'p-refine':
        kv_gs = gs.nurbs.gsKnotVector(kv_base, p_base)
        basis = gs.nurbs.gsBSplineBasis(kv_gs)
        if level > 0: basis.degreeElevate(level)
        p_cur = p_base + level
        title = f'p-refine (p={p_cur}, n={basis.size()})'
    else:  # k-refine
        kv_gs = gs.nurbs.gsKnotVector(kv_base, p_base)
        basis = gs.nurbs.gsBSplineBasis(kv_gs)
        if level > 0:
            basis.degreeElevate(level)
            basis.uniformRefine(level)
        p_cur = p_base + level
        title = f'k-refine (p={p_cur}, n={basis.size()})'

    n_sample = max(100, basis.size() * 8)
    params = np.linspace(0, 1, n_sample).reshape(1, -1)
    data = runge(-1 + 2 * params)

    fitter = gs.modelling.gsFitting(params, data, basis)
    fitter.compute()
    bspline = fitter.result()

    eval_pts = np.linspace(0, 1, 800).reshape(1, -1)
    x_eval = (-1 + 2 * eval_pts).flatten()
    y_bs = bspline.eval(eval_pts).flatten()
    y_exact = runge(x_eval)
    err = np.max(np.abs(y_exact - y_bs))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.plot(x_eval, y_exact, 'k-', lw=2, label='Runge function')
    ax1.plot(x_eval, y_bs, 'b-', lw=1.5, label='IGA fit')
    ax1.set_ylim(-0.15, 1.15)
    ax1.legend(); ax1.set_title(f'{title}\nerr = {err:.3e}')
    ax1.set_xlabel('$x$')

    # Basis functions
    kv_list = list(basis.knots(0).get())
    n_b = basis.size()
    xi = np.linspace(0, 1, 500).reshape(-1, 1)
    colors = plt.cm.tab10(np.linspace(0, 1, min(n_b, 10)))
    for i in range(min(n_b, 20)):  # Limit display
        cps_i = np.zeros((n_b, 1)); cps_i[i] = 1.0
        try:
            b = splinepy.BSpline(degrees=[p_cur], knot_vectors=[kv_list], control_points=cps_i)
            ax2.plot(xi, b.evaluate(xi), lw=1, color=colors[i % 10], alpha=0.7)
        except: pass
    ax2.set_title(f'Basis (p={p_cur}, n={n_b})')
    ax2.set_xlim(0, 1); ax2.set_ylim(-0.05, 1.1)

    plt.tight_layout(); plt.show()

widgets.interact(refinement_demo,
    strategy=widgets.Dropdown(options=['h-refine','p-refine','k-refine'], 
                              value='h-refine', description='Strategy'),
    level=widgets.IntSlider(min=0, max=5, value=2, description='Level'));

interactive(children=(Dropdown(description='Strategy', options=('h-refine', 'p-refine', 'k-refine'), value='h-…